In [2]:
import numpy as np
import cv2 as cv
import glob
import pyrealsense2 as rs

In [7]:
#get images
import cv2
import os
# Open the default camera
cap = cv2.VideoCapture(2)
for i in range(0,5):
    ret, frame = cap.read()
    save_path = 'images'
    filename = f'img{i}.jpg'
    cv2.putText(frame,f"Taking pic {i}",(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    output_path = os.path.join(save_path,filename)
    cv2.imshow('Hand Tracking',frame)
    cv2.imwrite(output_path, frame)
    cv2.waitKey(8000) #wait 8 seconds before taking picture
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [7]:
#getting object points and image points
x,y=6,9
chessboardSize = (x,y)
frameSize = (640,480)
objectPoints = np.zeros((6*9, 3), np.float32)
objectPoints[:, :2] = np.mgrid[0:6, 0:9].T.reshape(-1, 2)
objPoints = []
imgPoints = []
images = glob.glob(r'images\*.jpg')
print(images)
for image in images:
    img = cv.imread(image)
    print(img.shape)
    cv.waitKey(1000)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    cv.imshow('image',img)
    if ret == True:
        objPoints.append(objectPoints) #append object points created at the begining
        imgPoints.append(corners) #append the image points from the in built method
        cv.imshow('img',img)
cv.destroyAllWindows

['images\\img0.jpg', 'images\\img1.jpg', 'images\\img2.jpg', 'images\\img3.jpg', 'images\\img4.jpg']
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)


<function destroyAllWindows>

In [11]:
#getting calibration parameters
ret, camera_matrix, dist,rvecs, tvecs = cv.calibrateCamera(objPoints,imgPoints,frameSize,None,None)
print("Camera Calibrated:", ret)
print("Camera Matrix:\n", camera_matrix)
print("Distortion Parameters:\n",rvecs)
print("Rotation Vector:\n", rvecs)
print("Translation Vectors:\n", tvecs)
np.savez("camParameters",cameraMatrix=cameraMatrix,dist=dist,rvecs=rvecs,tvecs=tvecs)

Camera Calibrated: 0.19480214732563034
Camera Matrix:
 [[3.03417060e+03 0.00000000e+00 3.13155857e+02]
 [0.00000000e+00 3.03594823e+03 2.33492184e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 (array([[-0.08752895],
       [-0.06822106],
       [ 1.52994858]]), array([[-0.10141778],
       [-0.03935897],
       [ 0.96153075]]), array([[-0.13865789],
       [-0.06489543],
       [ 3.13036555]]), array([[ 0.03296858],
       [-0.01599872],
       [ 1.05228318]]), array([[-0.02829523],
       [-0.00765493],
       [ 1.51242111]]))
Rotation Vector:
 (array([[-0.08752895],
       [-0.06822106],
       [ 1.52994858]]), array([[-0.10141778],
       [-0.03935897],
       [ 0.96153075]]), array([[-0.13865789],
       [-0.06489543],
       [ 3.13036555]]), array([[ 0.03296858],
       [-0.01599872],
       [ 1.05228318]]), array([[-0.02829523],
       [-0.00765493],
       [ 1.51242111]]))
Translation Vectors:
 (array([[  4.14467236],
       [ -1.28067769],
       

NameError: name 'cameraMatrix' is not defined

In [13]:
#undistortion
img = cv.imread('images/img0.jpg')
height,width = img.shape[:2]
print(height,width)
newCameraMatrix, ret = cv.getOptimalNewCameraMatrix(camera_matrix,dist,(width,height),1,(width,height))

#undistort using cv.undistort
undst = cv.undistort(img,camera_matrix,dist,None,newCameraMatrix)
x,y,widht,height = ret
undst = undst[y:y+height,x:x+width] #cropping the image
cv.imwrite('images/testresult.png',undst)

480 640


True

In [18]:
camera_matrix[1][2]*2

466.9843676186273

In [5]:
import pyrealsense2 as rs
import numpy as np
import cv2

def main():
    # Configure RealSense pipeline
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

    # Start the pipeline
    pipeline.start(config)

    try:
        while True:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()

            if not depth_frame:
                continue

            # Convert depth frame to a numpy array
            depth_image = np.asanyarray(depth_frame.get_data())

            # Display depth image
            cv2.imshow('Depth Image', depth_image)

            # Exit the loop if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # Stop the pipeline and close all OpenCV windows
        pipeline.stop()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()